In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверяем, нет ли ошибок

        # Парсим HTML-код страницы
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем весь текст
        text = soup.get_text(separator='\n').replace('\xa0', ' ')

        return text.strip()
    except requests.exceptions.RequestException as e:
        return f"Ошибка при запросе к URL: {e}"


def extract_from_url(url, find):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверяем, нет ли ошибок

        # Парсим HTML-код страницы
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем текст из всех тегов <find> и очищаем от \xa0, \r, \n
        paragraphs = [
            p.get_text(strip=True)
            .replace('\xa0', ' ')
            .replace('\r', '')
            .replace('\n', '')
            for p in soup.find_all(find)
        ]

        return paragraphs
    except requests.exceptions.RequestException as e:
        return f"Ошибка при запросе к URL: {e}"

In [8]:
df = pd.read_csv('chunks.csv')


In [3]:
t = input()

 Университет Факультет ИУ Третий этаж ГЗ Деканат ИУ Телефон: 	+7-499-263-62-16 Телефон: 	+7-499-263-65-11  Декан: Пролетарский Андрей Викторович.  Часы приёма: День 	Время ПН 	нет приёма ВТ 	11:00 - 12:30 СР 	нет приёма ЧТ 	16:00 - 17:30 ПТ 	нет приёма СБ 	нет приёма ВС 	нет приёма Профком ИУ Девятый этаж ГЗ Кафедра ИУ5  Можно подняться по лестнице, но лучше на лифте. Заведующий кафедрой  Чёрненький Валерий Михайлович.  Кабинет: 902.  Часы приёма: День 	Время ПН 	нет приёма ВТ 	16:00 - 18:00 СР 	нет приёма ЧТ 	нет приёма ПТ 	16:00 - 18:00 СБ 	нет приёма ВС 	нет приёма Библиотека ГЗ УЛК Студсовет Поликлиника  Расписание приёма врачей. Бухгалтерия  Находится в корпусе поликлиники, на 4 этаже. Отдел по студенческим стипендиям  Кабинет 425.  График работы: День 	Время работы 	Обед ПН 	08:00 - 17:30 	12:30 - 13:30 ВТ СР ЧТ ПТ 	08:00 - 16:00 СБ 	выходной ВС Столовые ГЗ Первый этаж      студенческая столовая:         по будням: 10:00 до 18:00;         в субботу: с 09:00 до 17:00.     преподав

In [9]:
def split_text_into_chunks_with_overlap(text, chunk_size, overlap_size):
    """
    Разбивает текст на чанки заданного размера с пересечениями, сохраняя целостность слов.
    
    :param text: str, исходный текст
    :param chunk_size: int, максимальный размер одного чанка (в символах)
    :param overlap_size: int, размер пересечения между чанками (в символах)
    :return: list, список чанков
    """
    if not text:
        return []

    words = text.split()
    chunks = []
    current_chunk = []
    previous_chunk_end = []

    for word in words:
        # Проверяем, помещается ли слово в текущий чанк
        if sum(len(w) for w in current_chunk) + len(current_chunk) + len(word) <= chunk_size:
            current_chunk.append(word)
        else:
            # Добавляем текущий чанк в список чанков
            chunks.append(" ".join(current_chunk))
            # Сохраняем конец текущего чанка для пересечения
            previous_chunk_end = current_chunk[-overlap_size:] if overlap_size > 0 else []
            # Начинаем новый чанк с пересечением
            current_chunk = previous_chunk_end + [word]
    
    # Добавляем последний чанк
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [16]:
chunks = split_text_into_chunks_with_overlap(t, 500, 50)

In [23]:
len(chunks)

27

In [20]:
for name in df.columns:
    print(df.loc[43, name])

Размер стипендии составляет 15 000 рублей в месяцСтипендия от VK начисляется ежемесячно в течении 4 месяцев (сентябрь - декабрь 2023 для осеннего семестра; февраль 2024 - май 2024 для весеннего семестра)
Сколько месяцев будет выплачиваться стипендия от VK и какой размер этой стипендии?


Стипендия от VK начисляется ежемесячно в течение 4 месяцев: с сентября по декабрь 2023 года для осеннего семестра и с февраля по май 2024 года для весеннего семестра. Размер стипендии составляет 15 000 рублей в месяц.


In [21]:
chunks_f = list(df.chunks)
questions = list(df.questions)
answers = list(df.answers)

In [22]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML




INSTRUCTION = ("Я готовлю синтетические данные для РАГ. Придумай вопрос и ответ для чанка."
        "Вопрос должен быть похож на человеческий, как будто задаёт студент. Также вопрос должен содержать информацию, про что мы говорим."
        "Раздели вопрос и ответ только с помощью ---.")


sdk = YCloudML(
    folder_id = "b1gtjo1fd3s94mpcbgp4",
    auth="AQVNzuGi06FPMJ6VrcqKxfTJ-3G3ZGWX4rFfEDjN",
)


#каждый запуск - стоит денег, аккуратно
def generate_answer(inst, prompt, sdk=sdk, model = 'yandexgpt', temp = 0.7):
    messages = [
        {
            "role": "system",
            "text": inst,
        },
        {
            "role": "user",
            "text": prompt,
        },
    ]
    result = (
        sdk.models.completions("yandexgpt").configure(temperature=temp).run(messages)
    )
    return result[0].text
    

In [8]:
from_site = extract_from_url('https://ru.wikipedia.org/wiki/%D0%9C%D0%BE%D1%81%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D1%82%D0%B5%D1%85%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D1%83%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%B8%D0%BC%D0%B5%D0%BD%D0%B8_%D0%9D._%D0%AD._%D0%91%D0%B0%D1%83%D0%BC%D0%B0%D0%BD%D0%B0', 'p')


chunks = []

for text in from_site:
    if len(text)>100:
        chunks.append(text)



print(len(chunks))

49


In [24]:
chunks

['Университет Факультет ИУ Третий этаж ГЗ Деканат ИУ Телефон: +7-499-263-62-16 Телефон: +7-499-263-65-11 Декан: Пролетарский Андрей Викторович. Часы приёма: День Время ПН нет приёма ВТ 11:00 - 12:30 СР нет приёма ЧТ 16:00 - 17:30 ПТ нет приёма СБ нет приёма ВС нет приёма Профком ИУ Девятый этаж ГЗ Кафедра ИУ5 Можно подняться по лестнице, но лучше на лифте. Заведующий кафедрой Чёрненький Валерий Михайлович. Кабинет: 902. Часы приёма: День Время ПН нет приёма ВТ 16:00 - 18:00 СР нет приёма ЧТ нет',
 '16:00 - 17:30 ПТ нет приёма СБ нет приёма ВС нет приёма Профком ИУ Девятый этаж ГЗ Кафедра ИУ5 Можно подняться по лестнице, но лучше на лифте. Заведующий кафедрой Чёрненький Валерий Михайлович. Кабинет: 902. Часы приёма: День Время ПН нет приёма ВТ 16:00 - 18:00 СР нет приёма ЧТ нет приёма ПТ 16:00 - 18:00 СБ нет приёма ВС нет приёма Библиотека ГЗ УЛК Студсовет Поликлиника Расписание приёма врачей. Бухгалтерия Находится в корпусе поликлиники, на 4 этаже. Отдел по студенческим стипендиям Каби

In [25]:
chunks[-1]

'ул. Парковая 5-я, 30а (на карте) График работы: День Время работы Обед ПН 10:00 - 17:00 13:00 - 14:00 ВТ нет приёма - СР 10:00 - 17:00 13:00 - 14:00 ЧТ нет приёма - ПТ СБ ВС Для прохода в военкомат при себе нужно иметь паспорт или удостоверение гражданина, подлежащего призыву. А лучше и то, и то.'

In [26]:
generate_answer(INSTRUCTION, chunks[-1], model='yandexgpt',temp=0.5)

'Какой график работы у военкомата на 5-й Парковой, 30а?\n---\nВоенкомат работает с понедельника по среду с 10:00 до 17:00. Обеденный перерыв — с 13:00 до 14:00. Во вторник приёма нет.'

In [27]:
from tqdm import tqdm

In [28]:
triples = []
for ind, chunk in tqdm(enumerate(chunks)):
    t = generate_answer(INSTRUCTION, chunk, temp=0.5)
    t = t.split('---')
    if len(t)<2:
        t = t[0].split('—')
    if len(t)<2:
        t = t[0].split('Ответ:')
    if len(t)<2:
        break_ind = ind
        print(t)
        t.append(generate_answer('Сгенерируй ответ на вопрос', f'Вопрос: {t[0]}.\n\n Контекст: {chunk}'))

    triples.append((chunk, t[0], t[1]))
    questions.append(t[0])
    answers.append(t[1])
    chunks_f.append(chunk)

27it [00:45,  1.69s/it]


In [33]:
triples[15]

('курсов интернеты выдаются только при закрытой летней сессии, при себе иметь зачётку. График работы: День Время работы Обед ПН 10:00 - 16:00 13:00 - 14:00 ВТ СР ЧТ ПТ СБ выходной ВС Печать Где в университете можно распечатать документы, чертежи. Типография Находится на первом этаже ГЗ, там же, где выдают интернеты. Самые низкие цены в университете. График работы: День Время работы Обед ПН 10:00 - 17:00 - ВТ СР ЧТ ПТ СБ выходной ВС ЧЕРТЁЖ.ру ГЗ, 4 этаж, северное крыло. Контакты: сайт:',
 'Где в университете можно распечатать документы и получить интернет, и какие условия для этого нужно выполнить?\n',
 '\nЧтобы получить доступ к интернету и распечатать документы или чертежи в нашем университете, необходимо закрыть летнюю сессию. При себе нужно иметь зачётную книжку. Интернет выдают в месте, где можно распечатать документы — типография находится на первом этаже главного здания (ГЗ), там же, где выдают интернеты. Работает типография с понедельника по пятницу с 10:00 до 17:00, без обеда. 

In [30]:
print(len(chunks_f))
pd.DataFrame({'chunks':chunks_f, 'questions': questions, 'answers':answers}).to_csv('chunks.csv', index=False)

74


In [31]:
len(questions)

74